
# A State-Space Representation of the Fitness-Fatigue Model
In parts I-III of Modeling Cumulative Impact, the accumulating impact was modeled using convolution with either exponential decay or splines. Convolutions are sums, and can be used as convenient features for representing accumulating quatities. But the end of Part III hinted that there was a different approach to the accumulating quantity: *maintaining state*. 

$$
\text{E}(p_t) = 496 + 0.07 \sum_{i=1}^{n - 1} w_i \exp\left(\frac{-(n - i)}{\tau_1}\right) - 0.27 \sum_{i=1}^{n - 1} w_i \exp\left(\frac{-(n - i)}{\tau_2}\right).
$$

Let's denote  "fitness" at time t by $h_t = \sum_{i = 1}^{t - 1} w_i \exp(\frac{-(t - i)}{\tau_1})$ and seek its relationship to fitness one time period before it, $h_{t - 1} = \sum_{i = 1}^{ t - \color{red}2} w_i \exp(\frac{-(t - i \color{red}- \color{red}1)}{\tau_1}).$ Expanding the last term in the sum for $h_t$, we arrive at the recursion:
$$\begin{align*}
h_t &= \sum_{i = 1}^{t - 2} w_i \exp\left(\frac{-(t - i)}{\tau_1}\right) + w_{t - 1} \exp\left(\frac{-1}{\tau_1}\right) \\
    &= e^{-1 / \tau_1} h_{t - 1} + e^{-1 / \tau_1} w_{t - 1}.
\end{align*}$$
Since this is the same for fatigue $g_t$, the recursive relationship can be expressed using matrices in a "state-space form":

$$\left(\begin{matrix} 
h_t \\
g_t
\end{matrix}\right)=\left(\begin{matrix} 
e^{-1 / \tau_1}& 0 \\
0 & e^{-1 / \tau_2}
\end{matrix}\right)
\left(\begin{matrix} 
h_{t - 1} \\
g_{t - 1}
\end{matrix}\right) +
\left(\begin{matrix} 
e^{-1 / \tau_1} \\
e^{-1 / \tau_2}
\end{matrix}\right) w_{t - 1}.
$$

We can continue using matrices to express the second stage of the model with:
$$
p_t = \mu + 
\left(\begin{matrix} 
k_1 & 0 \\
0 & k_2
\end{matrix}\right)
\left(\begin{matrix} 
h_t \\
g_t
\end{matrix}\right)
+ r_t,
$$
where $r_t \sim N(0, \sigma^2).$ We have now arrived at a state-space formulation of the fitness-fatigue model. What does this get us? For one, it makes explicity the concept of evolving state. The dynamic formulas look cleaner than the ones with convolutions. But the real benefit, and the reason for this article, is that the state-space formulation makes available the Kalman Filter.



## The Kalman Filter

The world "Filter" in "Kalman Filter" refers to an operator that uses only the current and past observations. If an operator uses future observations as well, like a centered moving average, then in this context the operator is a "smoother" rather than a filter. "Kalman" comes from the name Rudolph Kalman, who's seminal 1960 paper [A New Approach to Linear Filtering and Prediction Problems](http://www.cs.unc.edu/~welch/kalman/kalmanPaper.html) earned him credit for the method. 

The following is one specification of the Kalman Filter model:

$$
\begin{align*}
x_t &= A x_{t - 1} + B u_{t - 1} +  q_t, &(1)\\
y_t &= H x_t + r_t,  &(2)
\end{align*}
$$

where $q_t \sim N(0, Q)$ and $r_t \sim N(0, R)$. There are variations on the above. Often the matrices $A, B, H, Q$ and $R$ are given time subscripts, and the external "control" input $+ B u_{t - 1}$ may be omitted or $+ B u_{t}$ may be used instead. Such distinctions matter little when working with the model analytically.

While Kalman's 1960 paper focuses on the general method for separating signal from noise, there [was an an almost simultaneous application](https://en.wikipedia.org/wiki/Kalman_filter#History) of the model to trajectory estimation. Indeed, the task motivates the model in a very intuitive way.

In trajectory estimation, the state space equation 1 is the model from Physics that specifies the dynamics of a moving body. The state vector may have position, velocity, for instance, and the transition matrix comes from the textbook theory. But as a batted baseball doesn't fly in a perfect parabola due to air pressure, etc., there will be "error" in the state space model. Suppose there is also a sensor tracking the baseball, then there is a sensor measurement represented by equation 2 that is a linear combination of the unknown state and also is affected by its own source of error. Combining or "fusing" the state space model's prediction with the observed measurement to get an improved estimate of the state is what the Kalman Filter does.

There's nobody (certainly not Kalman himself) saying that you have to use the Kalman Filter for trajectory estimation. You can use the Kalman Filter whenever you can formulate your problem into equations (1) and (2). This includes the ARIMA models, though their state space representations are not so intuitive. Still, the Kalman Filter representation has advantages, including straightforward imputation of missing values (just use the state space equation!), a convenient recursive updating equation, and easy access to the likelihood at each time point.

As mentioned, there are recursive updating equations for the Kalman Filter that provide:

1. The likelihood of the observed data each time point given current and past observations
2. The 1-step ahead state-space prediction
3. The "fused" estimate of the state-space prediction and the measurement


are complicated (more complicated than linear regression, at least) but straightforward

